In [67]:
import spacy
from spacy.matcher import Matcher
from IPython.display import Markdown, display
import os
import PyPDF2
import re
from IPython.display import clear_output
import xml.etree.ElementTree as ET
from spacy import displacy

In [2]:
spacy.prefer_gpu()
nlp = spacy.load("nl_core_news_lg")

base = r'F:\Data files\Master thesis\\'

# displacy.render(doc,style="ent",jupyter=True)

In [3]:
def printHilight(string):
    print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')

In [4]:
def extractText(base):
    if not os.path.isdir(base + 'text'):
        os.mkdir(base + 'text')
    
    for item in os.listdir(base):
        allText = ''
        
        
        if os.path.isdir(base + item):
            continue

        try:
            with open(base + item, 'rb') as f:
                reader = PyPDF2.PdfFileReader(f)

                for page in reader.pages:
                    text = page.extractText()
                    text = text.replace('\n', ' ')
                    
                    allText = allText + text + '\n'

        except:
            continue

        if len([x for x in allText.replace('\n', '').split(' ') if x]) > 10:
            with open("..\\data\\covid wob text without ocr\\" + item + '.txt' , 'w', encoding="utf-8") as f:
                f.write(allText)

In [8]:
def labelData():
    for file in os.listdir('..\\data\\covid wob text without ocr'):
        print(file)
        with open('..\\data\\covid wob text without ocr\\' + file, 'r', encoding='utf8') as f:
            text = f.read()

        text = re.sub(' +', ' ', text)
        text = text.strip()
        doc = nlp(text)
        
        listTokens = [str(ent.text) for ent in doc.ents]
        
        nToken = 0
        tempTokens = []
        lenTemp = 0
        
        print(doc[3:16])
        
        for token in doc:
            t = str(token)
            tempTokens.append((t, nToken))
            nToken += 1
            lenTemp += (len(t) + 1)

            if lenTemp >= 100:
                for word in tempTokens:
                    if len(word[0]) < len(str(word[1])):
                        print(' ' * (len(word[0]) + 1), end='')
                    else:
                        print(word[1], ' ' * (len(word[0]) - len(str(word[1]))), end='')
                print('')
                    
                for word in tempTokens:
                    if word[0] in listTokens:
                        printHilight(word[0])
                    else:
                        print(word[0] + ' ', end='')
                
                print('\n\n')
                tempTokens = []
                lenTemp = 0
                    
            

        return
        clear_output()
        


In [9]:
labelData()

0068ed0b40cca6270f857d2614cc63c0_besluit.pdf.txt
Korte Voorhout 7 2511 CW Den Haag Postbus 20201 2500 EE Den
0        1          2     3     4        5 6    7  8   9    10      11    12   13 14  15   16                   
Directie Juridische Zaken Korte Voorhout 7 2511 CW Den Haag Postbus 20201 2500 EE Den Haag www.rijksoverheid.nl 


17  18      19     21         22 23      25          28          29      30    31   32 33  34   35     
Ons kenmerk 2021 - 0000046938 Uw brief ( kenmerk ) > Retouradres Postbus 20201 2500 EE Den Haag Pagina 


  37    39    40 41    42   43      44      45 46 47    49      50     51       53        54      55 
1 van 3 Datum 25 maart 2021 Betreft Besluit op uw Wob - verzoek inzake agenda - afspraken Geachte In 


56 57    58  59 60   61     63        64 65 66   67     69      71  72  73     74 75 76  77           
uw brief van 10 juni 2020 , ontvangen op 15 juni 2020 , heeft u met een beroep op de Wet openbaarheid 


78  79        81       83    85       8

In [47]:
def loadTestData(path):
    with open(path, 'r', encoding='utf8') as f:
        text = f.read()
    
    text = text.split('\n')
    
    temp = []
    
    for x in text:
        x = x.split(' ')
        try:
            temp.append((x[0], x[1], x[2]))
        except:
            temp.append((None, None, None))
    
    text = ''
    
    for word in temp:
        if word[0] == None:
            text = text + '\n'
            
        elif word[0] in ',.!?':
            text = text + word[0]
        else:
            text = text + ' ' + word[0]
            
    return text, temp
    
text, labels = loadXML(r'..\\data\\ner labeled data\\test.tsv')
doc = nlp(text)
labels[:20]

[('-DOCSTART-', 'O', 'O'),
 ('Dat', 'O', 'O'),
 ('is', 'O', 'O'),
 ('in', 'O', 'O'),
 ('Italië', 'B-LOC', 'B-LOC'),
 (',', 'O', 'O'),
 ('Spanje', 'B-LOC', 'B-LOC'),
 ('of', 'O', 'O'),
 ('Engeland', 'B-LOC', 'B-LOC'),
 ('misschien', 'O', 'O'),
 ('geen', 'O', 'O'),
 ('probleem', 'O', 'O'),
 (',', 'O', 'O'),
 ('maar', 'O', 'O'),
 ('volgens', 'O', 'O'),
 ("'", 'O', 'O'),
 ('Der', 'B-PER', 'B-ORG'),
 ('Kaiser', 'I-PER', 'I-ORG'),
 ("'", 'O', 'O'),
 ('in', 'O', 'O')]

In [40]:
def printLabels(tempTokens):
        for word in tempTokens:
            if len(word[0][0]) < len(str(word[1])):
                print(' ' * (len(word[0][0]) + 1), end='')
            else:
                print(word[1], ' ' * (len(word[0][0]) - len(str(word[1]))), end='')
        print('')
        
        
        for word in tempTokens:
            if word[0][1] != 'O':
                printHilight(word[0][0])
            else:
                print(word[0][0] + ' ', end='')

        print('\n')

def showLabels(labels):
        nToken = 0
        tempTokens = []
        lenTemp = 0
        
        lineLimit = 50
        lines = 0
        
        for token in labels:
            if token[0] == None:
                lines += 1
                printLabels(tempTokens)
                tempTokens = []
                lenTemp = 0
            
            else:
                tempTokens.append((token, nToken))            
                nToken += 1
                lenTemp += (len(token[0]) + 1)


                if lenTemp >= 100:
                    lines += 1
                    printLabels(tempTokens)
                    tempTokens = []
                    lenTemp = 0
            
            
            if lines > lineLimit:
                return
            
showLabels(labels)

0          1   2  3  4      5 6      7  8        9         10   11         13   14        16  17     
-DOCSTART- Dat is in Italië , Spanje of Engeland misschien geen probleem , maar volgens ' Der Kaiser 

  19 20        21    
' in Duitsland wel . 

23     
sectie 

24     
Zürich 

25     
editie 

26   28       30      32      33 34        35       36          37 38  39        40 41          43     
De ' kleine ' clubs , waarvan de begroting dikwijls afhankelijk is van opleiding en transfers , klagen 

44    45 46   47   48  49          
steen en been over dat voornemen . 

51     52   53   54  55 56         57     58       59 60  61          
Andere zijn bang dat de salarissen straks helemaal de pan uitrijzen . 

63     
auteur 

64   65 66   67   68     69 70          71  72 73                 74 75         76  77  78    79    
Uefa en Fifa zijn bereid de afschaffing van de transfervergoeding te aanvaarden als dat enkel geldt 

80   81          82    83  84 85     
voor voetballers

In [51]:
def testNER(doc):
    labelsNER = []
    
    for token in doc[:1000]:
        if token.ent_type_:
            labelsNER.append((token.text, token.ent_type_))
            
        else:
            labelsNER.append((token.text, 'O'))
            
    return labelsNER
        
labelsNER = testNER(doc)
showLabels(labelsNER)

0 1          2   3  4  5      6 7      8  9        10        11   12         14   15        17  18     
  -DOCSTART- Dat is in Italië , Spanje of Engeland misschien geen probleem , maar volgens ' Der Kaiser 

  20 21        22    24 25     26 27     28 29     30 31   33       35      37      38 39        40       
' in Duitsland wel . 
  sectie 
  Zürich 
  editie 
  De ' kleine ' clubs , waarvan de begroting dikwijls 

41          42 43  44        45 46          48     49    50 51   52   53  54          56 57     58   
afhankelijk is van opleiding en transfers , klagen steen en been over dat voornemen . 
  Andere zijn 

59   60  61 62         63     64       65 66  67          69 70     71 72   73 74   75   76     77 78          
bang dat de salarissen straks helemaal de pan uitrijzen . 
  auteur 
  Uefa en Fifa zijn bereid de afschaffing 

79  80 81                 82 83         84  85  86    87    88   89          90    91  92 93     95 
van de transfervergoeding te aanvaarden als d

In [77]:
labelsEnts = [x for x in labels if x[1] != 'O' and x[1] != None]
l = ['FAC','GPE','LANGUAGE','LOC','NORP','ORG','PERSON','PRODUCT','WORK_OF_ART']
labelsNerEnts = [x.text for x in doc.ents if x.label_ in l]
len(labelsNerEnts), len(labelsEnts), doc.ents[1].text

(3344, 5758, 'Italië')

In [65]:
doc.ents[:100], labelsEnts[:100]

((-DOCSTART-, Italië, Spanje, Engeland, Der Kaiser, Duitsland, Zürich
   editie, 24 jaar, EU, Belgische, Europese, 24 jaar, 14, 24, 1 september 2000, Algemeen, Europese, bond, Europese Commissie, Reuters
   , Sepp Blatter, Zürich, achttien jaar, Franz Beckenbauer, Bayern München, 225 miljoen frank, 5,6 miljoen euro, -DOCSTART- Van Roost, Amerikaanse, eerste, 7, 6, 9, 7, tweede, 6, 4, Lindsay Davenport, twee, tweede, US Open, Kim Clijsters, Clijsters, Monami-Van Roost, New York, 1, 3, Belgische, eerste, 6, 4, 1 september 2000, Algemeen
   , Davenport, twee, 6, 2, 15, US Open, Clijsters, Davenport
   , -DOCSTART- pagina, LDP, Het zou een, Patrice D'Hoop, Franstalige, socialistische, bond FGTB, Vincent De Poortere, Johan Corthouts, JCS
   , Vincent De Poortere, LDP, Erik Coplo, De werknemers zijn het wachten beu, D'Hoop, Algemeen
   Domo, LDP, Vincent De Poortere, LDP, Balsan, 1 september 2000, Balsan, Franse, Châteauroux, maandag, Kortrijk, Moeskroen, vijfhonderd, LDP, Louis De Poortere,

In [61]:
tp = 0
tn = 0
fp = 0
fn = 0

indexNER, indexTest = 0,0

# for i in range(max(len(doc.ents), len(labelsEnts))):
for i in range(200):
    

In [75]:
for x in nlp.get_pipe("ner").labels:
    print(x, ' = ', spacy.explain(x))

CARDINAL  =  Numerals that do not fall under another type
DATE  =  Absolute or relative dates or periods
EVENT  =  Named hurricanes, battles, wars, sports events, etc.
FAC  =  Buildings, airports, highways, bridges, etc.
GPE  =  Countries, cities, states
LANGUAGE  =  Any named language
LAW  =  Named documents made into laws.
LOC  =  Non-GPE locations, mountain ranges, bodies of water
MONEY  =  Monetary values, including unit
NORP  =  Nationalities or religious or political groups
ORDINAL  =  "first", "second", etc.
ORG  =  Companies, agencies, institutions, etc.
PERCENT  =  Percentage, including "%"
PERSON  =  People, including fictional
PRODUCT  =  Objects, vehicles, foods, etc. (not services)
QUANTITY  =  Measurements, as of weight or distance
TIME  =  Times smaller than a day
WORK_OF_ART  =  Titles of books, songs, etc.


In [69]:
['FAC','GPE','LANGUAGE','LOC','NORP','ORG','PERSON','PRODUCT','WORK_OF_ART']

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [ ]:
                                                                                                                        D































